<a href="https://colab.research.google.com/github/MichaelTj02/FinalProject_Group1/blob/main/CombinedModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from ultralytics import YOLO
import cv2

# Combined model

In [ ]:
# Load YOLOv8 models
helmet_model = YOLO(r"C:\Users\mtj02\Desktop\FinalProject_Group1\Best_Motorcycle\weights\best.pt")
plate_model = YOLO(r"C:\Users\mtj02\Desktop\FinalProject_Group1\Best_LicensePlate\weights\best.pt")

In [ ]:
# Load the image
image = cv2.imread("path_to_image.jpg")